# BCI Comprehensive Data Alignment

# Section 1: Get epochs 

In [63]:
import mne
import numpy as np
from mne.datasets import eegbci
import matplotlib.pyplot as plt
from os import listdir
from mne.channels import make_standard_montage
from BCIAllFunction import BCIFuntions
from main_utilize import Unicorn

# ['Fz','C3', 'Cz','C4','Pz','PO7','Oz','PO8']

target_class = ["Left", "Right", "Non", "Feet"] 
AllBCIClass = Unicorn(selectclass = target_class, desired_fz = 128, ch_pick = ['Fz','C3','Cz','C4','Pz'])

target_data_0 = "pipo" #this subject will be test_set otherwise are train_set
calibrate_size = 80 # (trial)

condition_wLTL = "noEA"
train_svm = True

all_data = ["pipo","NutF8","AJpang","Aoomim","voen","pipo_HCI","Kawin"]

EEG_data = AllBCIClass.GetRawEDF(target_subjects= all_data, condition="Offline_Experiment") #Input data -> 250Hz with 6-32 Hz filtered + CAR

Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the instance is already 128.0, returning unmodified.
Sampling frequency of the

In [64]:
EEG_Epochs = AllBCIClass.GetEpoch(EEG_data ,tmin= -2.0, tmax= 6.0, crop = (0,4) ,baseline= (-0.5,0.0), band_pass=(6,32),trial_removal_th = 100)
calibrate_size = calibrate_size / EEG_Epochs[target_data_0]['Raw_Epoch'].shape[0]

Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
NutF8 -104.04128118288183 6
NutF8 86.93096401141673 6
NutF8 -104.31893713412965 11
NutF8 91.19854783269322 11
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']
voen -1044.3428689131774 1
voen 694.2336071556814 1
voen -1039.3090824149497 2
voen 1021.4912607644604 2
voen -952.1519349924243 3
voen 1248.5144240059017 3
voen -1608.9284132711775 4
voen 965.6585709158061 4
voen -1275.969071501411 5
voen 920.2

c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: T

pipo_HCI -55.97212970445327 79
pipo_HCI 116.32214024347675 79
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up']


c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()
c:\git\BCI_MI_Study\main_utilize.py:97: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  train_data = EEG_epoch[key_subs]['Raw_Epoch'].get_data()


# Section 2: Apply Data Alignment on RAW data

Note: After performing EA for all subjects, they share the same mean covariance matrix, i.e., the distances between the mean covariance matrices of different subjects are minimized (they become zero), and hence data distributions from different subjects become more similar.

In [65]:
alignmentMethod = "EA"

if alignmentMethod == "LA":
    AllBCIClass.ComputeLA(EEG_Epochs, target_subject= target_data_0, calibrate_size=calibrate_size)
    if calibrate_size != 0:
        target_data = target_data_0 + "_test"
    count = 0

    for index in range(len(EEG_Epochs[target_data]['KMediod_label'])):
        if EEG_Epochs[target_data]['label'][index] == EEG_Epochs[target_data]['KMediod_label'][index]:
            count += 1
    print(count/len(EEG_Epochs[target_data]['KMediod_label']) * 100)
    
else:
    AllBCIClass.ComputeEA(EEG_Epochs, target_subject= target_data_0, calibrate_size=calibrate_size)
    if calibrate_size != 0:
        target_data = target_data_0 + "_test"

In [66]:
target_data

'pipo_test'

# Section 3: vistualize raw data with and without EA

In [67]:
# if calibrate_size != 0:
#     AllBCIClass.plot_rawEA(EEG_Epochs, no_trial= 1, target_subject= target_data)

# Section 4 : CSP

In [68]:
CSP2D_Epoch = AllBCIClass.computeCSPFeatures(EEG_Epochs, target_subject = target_data)

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5


# Section 5 : T-SNE Plot

In [69]:
# AllBCIClass.TSNE_Plot(CSP2D_Epoch, target_subject= target_data)

# 6.1 CSP+LDA

In [70]:
AllBCIClass.classifyCSP_LDA(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0, condition = "noEA") #target_sub is used for target_data otherwise are source_data

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Classification TRAIN DATA 
              precision    recall  f1-score   support

        Left       0.38      0.55      0.45       202
       Right       0.31      0.35      0.33       200
         Non       0.37      0.31      0.33       202
        Feet       0.39      0.23      0.29       201

    a

In [71]:
AllBCIClass.classifyCSP_LDA(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0, condition = "EA")

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Classification TRAIN DATA 
              precision    recall  f1-score   support

        Left       0.45      0.52      0.48       202
       Right       0.43      0.39      0.41       200
         Non       0.40      0.37      0.38       202
        Feet       0.39      0.40      0.39       201

    a

# 6.2 CSP+SVM

In [72]:
if train_svm == True:
    AllBCIClass.classifyCSP_SVM(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0,condition = "noEA")

Computing rank from data with rank='info'


    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Best parameters: {'C': 1, 'kernel': 'rbf'}
Best cross-validation score: 0.381
Classification TRAIN DATA 
              precision    recall  f1-score   support

        Left       0.43      0.49      0.46       202
       Right       0.48      0.41      0.44       200
         Non       0.44      0.43      0.43       202
        Feet       0.48 

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
if train_svm == True:
    AllBCIClass.classifyCSP_SVM(EEG_Epochs, target_subjects= target_data, calibrate_data= target_data_0 ,condition = "EA")

Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank='info'
    MAG: rank 5 after 0 projectors applied to 5 channels
Reducing data rank from 5 -> 5
Estimating covariance using EMPIRICAL
Done.
Best parameters: {'C': 1, 'kernel': 'linear'}
Best cross-validation score: 0.406
Classification TRAIN DATA 
              precision    recall  f1-score   support

        Left       0.46      0.55      0.50       202
       Right       0.45      0.38      0.41       200
         Non       0.39      0.37

# 6.3 CSP+wLTL 

In [74]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# Custom loss function
class CustomLossLL1(tf.keras.losses.Loss):
    def __init__(self, lambda_t, model):
        super().__init__()
        self.lambda_t = lambda_t
        self.cross_entropy = CategoricalCrossentropy()
        self.model = model

    def call(self, y_true, y_pred):
        ce_loss = self.cross_entropy(y_true, y_pred)
        ws = self.get_weights_from_model()
        reg_term = self.regularization_term(ws)
        return ce_loss + self.lambda_t * reg_term

    def get_weights_from_model(self):
        model_weights = []
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                model_weights.append(layer.get_weights()[0])
        # return tf.concat([tf.reshape(w, [-1]) for w in model_weights], axis=0)
        return model_weights

    def regularization_term(self, ws):
        reg_term = tf.pow(tf.norm(ws, ord='euclidean'),2)
        return reg_term


# Custom training loop
def custom_train_step(model, optimizer, x, y, custom_loss):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True) # Perform a forward pass and compute the predictions
        loss = custom_loss(y, y_pred) # Compute the custom loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def train_weight_LL(X_train, y_train, lambd, num_tier=10, learning_rate = 0.01):
    n_classes = np.unique(y_train).size
    _, sequential_indices = np.unique(y_train, return_inverse=True)
    y_one_hot = tf.keras.utils.to_categorical(sequential_indices, num_classes=n_classes)

    lambda_t = lambd  # Regularization parameter

    model = Sequential([
        Dense(n_classes, input_shape=(X_train.shape[1],), activation='softmax')  # Adjust input_shape to match the number of features in X
    ])

    # Compile the model
    optimizer = Adam(learning_rate)
    custom_loss = CustomLossLL1(lambda_t, model)

    # Custom training loop
    epochs = num_tier
    lowest_loss = float('inf')
    best_weights = None
    for epoch in range(epochs):
        loss = custom_train_step(model, optimizer, X_train, y_one_hot, custom_loss)
        loss_value = loss.numpy()
        if loss_value < lowest_loss:
            lowest_loss = loss_value
            best_weights = [layer.get_weights() for layer in model.layers]

        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

    # best_weights = [layer.get_weights() for layer in model.layers]

    return best_weights, lowest_loss

def build_clf_params(data, target_subjects ,condition):

    for sub in data.keys():
        if (sub  != target_subjects) and  (sub != target_data_0): #Don't apply weight to target subject
            # Where the tranining data is stored
            if condition == "noEA":
                X = data[sub]['Raw_csp']
                y = data[sub]['Raw_csp_label']
                store_ws = 'ws_Raw'

            else:
                X = data[sub]['EA_csp']
                y = data[sub]['EA_csp_label']
                store_ws = 'ws_EA'

            weights, loss = train_weight_LL(X_train=X, y_train=y, lambd= 0.1, num_tier=500, learning_rate= 0.01)
            print("weights of ", str(sub), ": ", weights)
            print("Lowest loss of ", str(sub), ": ", loss)
            data[sub][store_ws] = weights

build_clf_params(CSP2D_Epoch, target_subjects= target_data ,condition = condition_wLTL)

Epoch 0, Loss: 1.9907100200653076
Epoch 20, Loss: 1.7840031385421753
Epoch 40, Loss: 1.7137718200683594
Epoch 60, Loss: 1.6891484260559082
Epoch 80, Loss: 1.6981854438781738
Epoch 100, Loss: 1.7261500358581543
Epoch 120, Loss: 1.7595738172531128
Epoch 140, Loss: 1.791982889175415
Epoch 160, Loss: 1.822627305984497
Epoch 180, Loss: 1.8515746593475342
Epoch 200, Loss: 1.8788220882415771
Epoch 220, Loss: 1.9042681455612183
Epoch 240, Loss: 1.9278955459594727
Epoch 260, Loss: 1.9497299194335938
Epoch 280, Loss: 1.9698561429977417
Epoch 300, Loss: 1.9884016513824463
Epoch 320, Loss: 2.00551700592041
Epoch 340, Loss: 2.021357536315918
Epoch 360, Loss: 2.036071538925171
Epoch 380, Loss: 2.049793004989624
Epoch 400, Loss: 2.06264066696167
Epoch 420, Loss: 2.0747153759002686
Epoch 440, Loss: 2.086101770401001
Epoch 460, Loss: 2.096871852874756
Epoch 480, Loss: 2.1070847511291504
weights of  NutF8 :  [[array([[ 0.69943035,  0.10467657,  0.4449077 ,  0.62720466],
       [ 0.6684168 ,  0.09534825,

NOTE:
Weights (array): These are the parameters that the model learns during training. They represent the strength of the connections between neurons. The shape of the weights array is (input_dim, output_dim).

Biases (array): These are additional parameters that are added to the outputs of each layer. The shape of the biases array is (output_dim,).

In [75]:
CSP2D_Epoch[target_data].keys()

dict_keys(['Raw_csp', 'Raw_csp_label', 'EA_csp', 'EA_csp_label'])

In [76]:
# First define the kl divergence
def KL_div(P, Q):
    # First convert to np array
    P = np.array(P)
    Q = np.array(Q)
    
    # Then compute their means, datain shape of samples x feat
    mu_P = np.mean(P, axis=0)
    mu_Q = np.mean(Q, axis=0)    

    
    # Compute their covariance
    sigma_P = np.cov(P, rowvar=False)
    sigma_Q = np.cov(Q, rowvar=False)  

    diff = mu_Q - mu_P

    inv_sigma_Q = np.linalg.inv(sigma_Q)
    term1 = np.dot(np.dot(diff.T, inv_sigma_Q), diff)
    
    # Calculate the trace term trace(Sigma_Q^{-1} * Sigma_P)
    term2 = np.trace(np.dot(inv_sigma_Q, sigma_P))
    
    # Calculate the determinant term ln(det(Sigma_P) / det(Sigma_Q))
    det_sigma0 = np.linalg.det(sigma_P)
    det_sigma1 = np.linalg.det(sigma_Q)

    
    epsilon = 1e-6
    term3 = np.log((det_sigma0+epsilon) / (det_sigma1+epsilon))
    
    print(term3)
    
    # Dimensionality of the data
    K = mu_P.shape[0]
    
    # KL divergence
    kl_div = 0.5 * (term1 + term2 - term3 - K)
    
    return kl_div

In [77]:
# Compute kl divergence of target subject to each source subject
def compute_all_kl_div(data, target_subjects , condition):
    '''
    Parameter:
    data, is the whole data containing target and source data
    '''
    kl_div_score = []

    if condition == "noEA":
        target_data = 'Raw_csp'
        label_name = 'Raw_csp_label'

    else:
        target_data = 'EA_csp'
        label_name = 'EA_csp_label'
        
    # cal P from target data
    label_tgt =  data[target_subjects][label_name]
    P_left =  data[target_subjects][target_data][np.where(label_tgt == 0)]
    P_right = data[target_subjects][target_data][np.where(label_tgt == 1)]
    P_non = data[target_subjects][target_data][np.where(label_tgt == 2)]
    P_feet = data[target_subjects][target_data][np.where(label_tgt == 3)]

    tgt_data = target_subjects + "_test"

    #cal Q from each source subject
    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            label_src =  data[sub][label_name]
            Q_left =  data[sub][target_data][np.where(label_src == 0)]
            Q_right = data[sub][target_data][np.where(label_src == 1)]
            Q_non = data[sub][target_data][np.where(label_src == 2)]
            Q_feet = data[sub][target_data][np.where(label_src == 3)]

            kl_left = KL_div(P_left, Q_left)
            kl_right = KL_div(P_right, Q_right)
            kl_non = KL_div(P_non, Q_non)
            kl_feet = KL_div(P_feet, Q_feet)

            kl_div_temp = [kl_left, kl_right, kl_non, kl_feet]

            kl_div_score.append(kl_div_temp)

    data[target_subjects]['kl_div'] = kl_div_score


compute_all_kl_div(CSP2D_Epoch, target_subjects=target_data_0 ,condition = condition_wLTL) #target_sub for cal KL is calibrate set

-1.3046770928094429
-0.30313800831733073
-0.3956192787448664
0.12879469476507202
0.36642578263452724
0.11106366462871675
0.4395169503165339
0.2499712563543412
0.34141913578085487
0.09826709477295835
0.32016139349977046
0.145372676226133
-0.07811140654969706
-0.07603681956520206
0.36610202802832825
0.21080000037352828
0.0327858806979166
-0.2657393129127305
-0.5695613042016362
0.012880758601432106
0.07732334004047996
-0.17556017824454417
0.22941825284682085
-0.11652401440392393


In [78]:
np.array(CSP2D_Epoch[target_data_0]['kl_div'])

array([[ 47.98235552,  77.3001835 , 100.18540757, 201.00360609],
       [130.99981818, 209.72691605, 139.41718465, 244.78983674],
       [ 64.9535467 ,  66.44475356,  59.71977882, 116.68759599],
       [ 15.21296407,  27.28200274,  31.31743398,  66.05890354],
       [ 20.68926923,  15.95620305,  15.81935952,  15.3332422 ],
       [ 22.65021433,  22.42584721,  26.14708848,  16.88023443]])

In [79]:
def compute_similarity_weights(data, target_subjects):
    kl = data[target_subjects]['kl_div']
    KL_inv_left = []
    KL_inv_right = []
    KL_inv_non = []
    KL_inv_feet = []

    alpha_s = []
    eps = 0.0001
    
    #equation (9)
    for val in kl:
        if val != 0: 
            KL_inv_left.append(1/((val[0] + eps)**4))
            KL_inv_right.append(1/((val[1] + eps)**4))
            KL_inv_non.append(1/((val[2] + eps)**4))
            KL_inv_feet.append(1/((val[3] + eps)**4))

    print(KL_inv_left)
    print(KL_inv_right)
    
    for i in range(0,len(KL_inv_left)):
        temp = [KL_inv_left[i]/sum(KL_inv_left), KL_inv_right[i]/sum(KL_inv_right), KL_inv_non[i]/sum(KL_inv_non), KL_inv_feet[i]/sum(KL_inv_feet)]
        alpha_s.append(temp)

    alpha_s = np.array(alpha_s)
    print(np.array(alpha_s[:, ~np.isnan(alpha_s).any(axis=0)]))
                
    data[target_subjects]['alpha_s'] = alpha_s[:, ~np.isnan(alpha_s).any(axis=0)]

compute_similarity_weights(CSP2D_Epoch, target_subjects=target_data_0)

[1.8865578300743208e-07, 3.3955951325675196e-09, 5.6180531693435696e-08, 1.866952301447292e-05, 5.457716834220984e-06, 3.7992954400499027e-06]
[2.800758438881444e-08, 5.168713840362148e-10, 5.130439914067606e-08, 1.8050475207749819e-06, 1.542662406030965e-05, 3.9536372313637e-06]
[[6.69591275e-03 1.31706575e-03 5.15976987e-04 2.01082633e-05]
 [1.20519013e-04 2.43060446e-05 1.37588920e-04 9.14148148e-06]
 [1.99400163e-03 2.41260602e-03 4.08671424e-03 1.77047813e-04]
 [6.62632734e-01 8.48829455e-02 5.40380556e-02 1.72370424e-03]
 [1.93709385e-01 7.25442003e-01 8.30010099e-01 5.93804537e-01]
 [1.34847447e-01 1.85921074e-01 1.11211565e-01 4.04265461e-01]]


In [80]:
def compute_ETL_and_mu_ws(data, target_subjects, condition):

    mu_ws = 0
    temp_ws = 0

    if condition == "noEA":
        ws_name = 'ws_Raw'
    else:
        ws_name = 'ws_EA'

    alpha_s = np.array(data[target_subjects]['alpha_s'])

    tgt_data = target_subjects + "_test"
    index_count = 0

    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            ws = data[sub][ws_name][0][0]
            # mu_ws += ws @ alpha_s  #equation (10)
            # mu_ws += np.dot(ws, np.transpose(alpha_s))
            mu_ws += ws * alpha_s[index_count]
            index_count += 1

    print(np.array(mu_ws))

    index_count = 0
    for sub in data.keys():
        if (sub != target_subjects) and (sub != tgt_data):
            ws = data[sub][ws_name][0][0]
            # ws_min_mu = np.dot((np.dot(ws,np.transpose(alpha_s)) - mu_ws) , np.transpose((np.dot(ws,np.transpose(alpha_s)) - mu_ws)))
            ws_min_mu = np.dot(((ws * alpha_s[index_count]) - mu_ws), np.transpose((ws * alpha_s[index_count]) - mu_ws))
            temp_ws += ws_min_mu #equation (11)
            index_count += 1

    print(np.array(temp_ws))
    
    # den = np.diag(temp_ws) #get array in diagonal line

    den = temp_ws
    nom = np.trace(temp_ws) #Return the sum along diagonals of the array.
    Sigma_TL = den/nom


    data[target_subjects]['Sigma_TL'] = Sigma_TL
    data[target_subjects]['mu_ws'] = mu_ws

compute_ETL_and_mu_ws(CSP2D_Epoch, target_subjects = target_data_0, condition=condition_wLTL)

[[-0.56963538 -0.07891807 -0.41193258 -0.23700109]
 [ 0.78274249  0.13416051 -0.56520589 -0.17102968]
 [-0.33783114 -0.33026183  0.5302145   0.23263494]
 [-0.14598631 -0.32176081  0.00305676 -0.10510545]
 [ 0.2287925   0.86655066  0.12515315  0.11369006]]
[[ 2.68236572 -0.7282491  -0.42109064  0.57807879 -1.21808628]
 [-0.7282491   4.73896302 -3.13848736 -0.72429017  0.99904081]
 [-0.42109064 -3.13848736  2.76197986  0.65866076 -1.31708557]
 [ 0.57807879 -0.72429017  0.65866076  0.65936736 -1.55546182]
 [-1.21808628  0.99904081 -1.31708557 -1.55546182  4.04919775]]


In [81]:
print(np.array(CSP2D_Epoch[target_data_0]['Sigma_TL']).shape)
print(np.array(CSP2D_Epoch[target_data_0]['mu_ws']).shape)

(5, 5)
(5, 4)


# Test ACC with target subject

In [82]:
# Custom loss function
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

class CustomLossLL2(tf.keras.losses.Loss):
    def __init__(self, lambda_t, model, mu, sigma_TL):
        super().__init__()
        self.lambda_t = lambda_t
        self.cross_entropy = CategoricalCrossentropy()
        self.model = model
        self.mu = tf.convert_to_tensor(mu, dtype=tf.float32)
        self.sigma_TL = tf.convert_to_tensor(sigma_TL, dtype=tf.float32)

    def call(self, y_true, y_pred):
        ce_loss = self.cross_entropy(y_true, y_pred)
        wt = self.get_weights_from_model()
        reg_term = self.regularization_term(wt)

        return ce_loss + (self.lambda_t * tf.linalg.matmul(reg_term, wt))

    def get_weights_from_model(self):
        model_weights = []
        for layer in self.model.layers:
            if len(layer.get_weights()) > 0:
                model_weights.append(layer.get_weights()[0])
        # return tf.concat([tf.reshape(w, [-1]) for w in model_weights], axis=0)
        return model_weights

    def regularization_term(self, wt):
        diff = wt - self.mu
        reg_term = 0.5 * tf.linalg.matmul(tf.linalg.matmul(tf.linalg.inv(self.sigma_TL), diff[0]), tf.transpose(diff[0]))
        reg_term += 0.5 * tf.math.log(tf.linalg.det(self.sigma_TL))
        return reg_term


# Custom training loop
def custom_train_step(model, optimizer, x, y, custom_loss):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True) # Perform a forward pass and compute the predictions
        loss = custom_loss(y, y_pred) # Compute the custom loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


def train_weight_LL2(X_train, y_train, lambd, mu, sigma_TL, num_tier=10, learning_rate = 0.01):
    n_classes = np.unique(y_train).size
    _, sequential_indices = np.unique(y_train, return_inverse=True)
    y_one_hot = tf.keras.utils.to_categorical(sequential_indices, num_classes=n_classes)

    lambda_t = lambd  # Regularization parameter

    model = Sequential([
        Dense(n_classes, input_shape=(X_train.shape[1],), activation='softmax')  # Adjust input_shape to match the number of features in X
    ])

    # Compile the model
    optimizer = Adam(learning_rate)
    custom_loss = CustomLossLL2(lambda_t, model, mu, sigma_TL)

    # Custom training loop
    epochs = num_tier
    lowest_loss = float('inf')
    best_weights = None
    best_model = None

    for epoch in range(epochs):
        loss = custom_train_step(model, optimizer, X_train, y_one_hot, custom_loss)
        loss_value = loss.numpy()
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

        if (abs(loss_value) < lowest_loss):
            lowest_loss = abs(loss_value)
            best_model = model
            best_weights = [layer.get_weights() for layer in model.layers]

    return best_model, best_weights, lowest_loss

def GetConfusionMatrix(model, X_train, X_test, y_train, y_test):
    y_pred_prob = model.predict(X_train)
    y_pred = np.argmax(y_pred_prob, axis=1)
    _, y_train = np.unique(y_train, return_inverse=True)

    print("Classification TRAIN DATA \n=======================")
    print(classification_report(y_true= y_train, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true= y_train, y_pred=y_pred))

    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    _, y_test = np.unique(y_test, return_inverse=True)

    print("Classification TEST DATA \n=======================")
    print(classification_report(y_true=y_test, y_pred=y_pred))
    print("Confusion matrix \n=======================")
    print(confusion_matrix(y_true=y_test, y_pred=y_pred))


def tgt_test_wLTL(data, target_subjects ,condition):
    tgt_data = target_subjects + "_test"

    if condition == "noEA":
        X = data[target_subjects]['Raw_csp']
        y = data[target_subjects]['Raw_csp_label']
        X_test = data[tgt_data]['Raw_csp']
        y_test = data[tgt_data]['Raw_csp_label']
        store_ws = 'wt_Raw'

    else:
        X = data[target_subjects]['EA_csp']
        y = data[target_subjects]['EA_csp_label']
        X_test = data[tgt_data]['EA_csp']
        y_test = data[tgt_data]['EA_csp_label']
        store_ws = 'wt_EA'

    mu = data[target_subjects]['mu_ws']
    sigma_TL = data[target_subjects]['Sigma_TL']

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state = 42, stratify=y)

    X_train = X
    y_train = y
    
    model, weights, loss = train_weight_LL2(X_train=X_train, y_train=y_train, mu =mu, sigma_TL=sigma_TL,  lambd= 0.1, num_tier=3000, learning_rate= 0.005)
    print("weights of ", str(target_subjects), ": ", weights)
    print("loss of ", str(target_subjects), ": ", loss)
    data[target_subjects][store_ws] = weights

    GetConfusionMatrix(model, X_train, X_test, y_train, y_test)

tgt_test_wLTL(CSP2D_Epoch, target_subjects= target_data_0 ,condition = condition_wLTL)

Epoch 0, Loss: 3.408442735671997
Epoch 20, Loss: 4.188305854797363
Epoch 40, Loss: 4.640628814697266
Epoch 60, Loss: 5.1712236404418945
Epoch 80, Loss: 5.953761100769043
Epoch 100, Loss: 7.131471157073975
Epoch 120, Loss: 8.748147010803223
Epoch 140, Loss: 10.88715648651123
Epoch 160, Loss: 13.598322868347168
Epoch 180, Loss: 16.937984466552734
Epoch 200, Loss: 20.959732055664062
Epoch 220, Loss: 25.71445655822754
Epoch 240, Loss: 31.249401092529297
Epoch 260, Loss: 37.607696533203125
Epoch 280, Loss: 44.82745361328125
Epoch 300, Loss: 52.94129180908203
Epoch 320, Loss: 61.97607421875
Epoch 340, Loss: 71.9527587890625
Epoch 360, Loss: 82.88658142089844
Epoch 380, Loss: 94.78746795654297
Epoch 400, Loss: 107.65995025634766
Epoch 420, Loss: 121.504150390625
Epoch 440, Loss: 136.31591796875
Epoch 460, Loss: 152.0874481201172
Epoch 480, Loss: 168.8076629638672
Epoch 500, Loss: 186.4626007080078
Epoch 520, Loss: 205.03628540039062
Epoch 540, Loss: 224.5105743408203
Epoch 560, Loss: 244.8659